# 2. Análise Estatística dos Dados PISA

Este notebook realiza análises estatísticas aprofundadas dos dados do PISA.

## Objetivos
1. Realizar testes de normalidade
2. Analisar diferenças entre grupos
3. Investigar relações entre variáveis
4. Identificar fatores significativos
5. Criar visualizações estatísticas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd

%matplotlib inline
plt.style.use('seaborn')
pd.set_option('display.max_columns', None)

## 1. Carregamento e Preparação dos Dados

In [ ]:
# Carrega dados
df = pd.read_csv('../data/economics_and_education_dataset_CSV.csv')

# Remove linhas com valores ausentes nas variáveis principais
main_vars = ['rating', 'expenditure_on _education_pct_gdp', 'gdp_per_capita_ppp', 'gini_index']
df_clean = df.dropna(subset=main_vars)

print('Shape após limpeza:', df_clean.shape)

## 2. Testes de Normalidade

In [ ]:
def test_normality(data, var_name):
    """Realiza teste de normalidade e cria Q-Q plot."""
    # Teste de Shapiro-Wilk
    stat, p_value = stats.shapiro(data)
    
    # Q-Q Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    # Histograma
    sns.histplot(data=data, ax=ax1)
    ax1.set_title(f'Distribuição de {var_name}')
    
    # Q-Q Plot
    stats.probplot(data, dist="norm", plot=ax2)
    ax2.set_title('Q-Q Plot')
    
    plt.tight_layout()
    plt.show()
    
    print(f'Teste de Normalidade para {var_name}:')
    print(f'Estatística: {stat:.4f}')
    print(f'P-valor: {p_value:.4f}')
    print('Conclusão:', 'Normal' if p_value > 0.05 else 'Não Normal')
    print('---')

# Testa normalidade das principais variáveis
for var in main_vars:
    test_normality(df_clean[var], var)

## 3. Análise de Diferenças entre Grupos

In [ ]:
# ANOVA: Diferenças entre países
f_stat, p_val = stats.f_oneway(*[group['rating'].values for name, group in df_clean.groupby('country')])

print('ANOVA - Diferenças entre Países')
print(f'F-statistic: {f_stat:.4f}')
print(f'P-valor: {p_val:.4f}')

# Teste t: Diferenças entre sexos
boys = df_clean[df_clean['sex'] == 'BOY']['rating']
girls = df_clean[df_clean['sex'] == 'GIRL']['rating']
t_stat, p_val = stats.ttest_ind(boys, girls)

print('\nTeste t - Diferenças entre Sexos')
print(f'T-statistic: {t_stat:.4f}')
print(f'P-valor: {p_val:.4f}')

## 4. Análise de Correlação e Regressão

In [ ]:
# Matriz de correlação com p-valores
def correlation_with_pval(data, vars_to_analyze):
    correlations = pd.DataFrame(index=vars_to_analyze, columns=vars_to_analyze)
    p_values = pd.DataFrame(index=vars_to_analyze, columns=vars_to_analyze)
    
    for var1 in vars_to_analyze:
        for var2 in vars_to_analyze:
            corr, p_val = stats.pearsonr(data[var1], data[var2])
            correlations.loc[var1, var2] = corr
            p_values.loc[var1, var2] = p_val
    
    return correlations, p_values

# Calcula correlações e p-valores
corr_matrix, p_matrix = correlation_with_pval(df_clean, main_vars)

print('Matriz de Correlação:')
print(corr_matrix.round(3))
print('\nMatriz de P-valores:')
print(p_matrix.round(3))

## 5. Análise de Tendências

In [ ]:
# Análise de tendência temporal
yearly_means = df_clean.groupby('time')['rating'].mean()

# Teste de tendência
slope, intercept, r_value, p_value, std_err = stats.linregress(range(len(yearly_means)), yearly_means)

# Plota tendência
plt.figure(figsize=(10, 6))
plt.plot(yearly_means.index, yearly_means.values, 'o-')
plt.plot(yearly_means.index, intercept + slope * np.arange(len(yearly_means)), 'r--')
plt.title('Tendência Temporal das Notas PISA')
plt.xlabel('Ano')
plt.ylabel('Nota Média')
plt.grid(True)

print('Análise de Tendência:')
print(f'Inclinação: {slope:.4f}')
print(f'R²: {r_value**2:.4f}')
print(f'P-valor: {p_value:.4f}')

## 6. Análise de Variância Multifatorial

In [ ]:
# Categoriza variáveis contínuas
df_clean['gdp_level'] = pd.qcut(df_clean['gdp_per_capita_ppp'], q=3, labels=['Low', 'Medium', 'High'])
df_clean['education_spending_level'] = pd.qcut(df_clean['expenditure_on _education_pct_gdp'], q=3, labels=['Low', 'Medium', 'High'])

# ANOVA multifatorial
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols

model = ols('rating ~ C(gdp_level) + C(education_spending_level) + C(gdp_level):C(education_spending_level)',
            data=df_clean).fit()
anova_table = anova_lm(model, typ=2)

print('ANOVA Multifatorial:')
print(anova_table)

# Visualiza interações
plt.figure(figsize=(10, 6))
sns.boxplot(x='gdp_level', y='rating', hue='education_spending_level', data=df_clean)
plt.title('Interação entre PIB e Gastos em Educação')
plt.show()